# A small fine-tuning experiment with OpenAI

In [1]:
!pip install python-dotenv numpy pandas matplotlib huggingface_hub datasets openai

Looking in indexes: https://pypi.org/simple, https://****@pypi.fury.io/sarus/


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from datasets import load_dataset

dataset = load_dataset("LeoTungAnh/electricity_hourly")

In [4]:
dataset['train']

Dataset({
    features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
    num_rows: 370
})

In [ ]:
# import os
# from huggingface_hub import login

# login(os.getenv("HF_TOKEN"))
# dataset = load_dataset("EDS-lab/electricity-demand", "demand")

In [23]:
from json import dumps

def format(series):
    return dumps({"messages": [
        {"role": "system", "content": "Given a meter ID, you return a series of hourly consumptions given as a json string."},
        {"role": "user", "content": dumps({'item_id': series['item_id']})},
        {"role": "assistant", "content": dumps({'consumption': [round(1000*x)/1000 for x in series['target'][:100]]})}
    ]})+"\n"

with open('/tmp/finetuning_train.jsonl', 'w') as f:
    for series in dataset['train']:
        f.write(format(series))

with open('/tmp/finetuning_test.jsonl', 'w') as f:
    for series in dataset['test']:
        f.write(format(series))

In [16]:
print(training_data[1])

{'messages': [{'role': 'system', 'content': 'Given a meter ID, you return a series of hourly consumption given as a json string.'}, {'role': 'user', 'content': '{"item_id": "MT_002"}'}, {'role': 'assistant', 'content': '{"consumption": [-0.7670478012651036, -0.711666063830166, -0.7393569325476368, -0.8778112761349713, -0.9055021448524383, -0.8778112761349713, -0.9055021448524383, -0.8501204074175045, -0.8778112761349713, -0.5178299828078886, 0.22982347256373997, 1.3651490899799106, 1.863584726894325, 1.060549534087766, 0.17444173512880556, -0.07477608332841282, -0.24092129563321438, -0.18553955819828052, -0.24092129563321438, -0.26861216435068186, 0.11905999769386784, -0.13015782076334279, -0.29630303306814876, -0.6285934576777608, -0.7670478012651036, -0.9055021448524383, -0.9608838822873728, -0.9885747510048396, -0.9331930135699058, -0.8778112761349713, -0.7116660638301654, -0.019394345893474534, 0.14675086641133478, 0.20213260384627305, 0.5621138971733441, 0.4790412910209434, 0.3405